# Keras Digit Recogniser

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Flatten
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

In [ ]:
df_data_1 = pd.read_csv('train.csv')
df_data_2 = pd.read_csv(test.csv)
train = df_data_1
test = df_data_2.values.astype('float32')

In [46]:
# preview data
train.head()
#test.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
labels = train.label.values.astype('int32')
train_img = train.loc[:,'pixel0':].values.astype('float32')
#labels
train_img.shape, test.shape
#type(labels)

((42000, 784), (28000, 784))

In [48]:
# convert labels to a categorical data type, as expected by keras model
labels = to_categorical(labels)
classes = labels.shape[1]
labels

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [49]:
# fix random seed for reproducibility
seed = 43
np.random.seed(seed)

## Multilayer Perceptron

Now we will build the neural network. We use a Keras Sequential object to build a simple feedforward network (multilayer perceptron, or MLP), with Relu input and hidden layer activations, and a softmax output layer with 10 classes. model.add() will be used to add layers.

In [50]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=784))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [51]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [52]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model_fit = model.fit(train_img, labels, epochs=100, batch_size=64, validation_split=0.1, callbacks=[early_stopping])

Train on 37800 samples, validate on 4200 samples
Epoch 1/100
37800/37800 [==============================] - 8s 207us/step - loss: 0.6875 - acc: 0.7789 - val_loss: 0.3188 - val_acc: 0.9117
Epoch 2/100
37800/37800 [==============================] - 7s 194us/step - loss: 0.2725 - acc: 0.9224 - val_loss: 0.2909 - val_acc: 0.9171
Epoch 3/100
37800/37800 [==============================] - 7s 197us/step - loss: 0.2162 - acc: 0.9403 - val_loss: 0.2351 - val_acc: 0.9331
Epoch 4/100
37800/37800 [==============================] - 7s 194us/step - loss: 0.1847 - acc: 0.9489 - val_loss: 0.2072 - val_acc: 0.9455
Epoch 5/100
37800/37800 [==============================] - 7s 195us/step - loss: 0.1611 - acc: 0.9543 - val_loss: 0.2101 - val_acc: 0.9467
Epoch 6/100
37800/37800 [==============================] - 7s 196us/step - loss: 0.1429 - acc: 0.9600 - val_loss: 0.1936 - val_acc: 0.9495
Epoch 7/100
37800/37800 [==============================] - 7s 194us/step - loss: 0.1348 - acc: 0.9642 - val_loss: 0.1

In [53]:
pred = model.predict_classes(test)
pred, len(pred)

(array([2, 0, 9, ..., 3, 9, 2]), 28000)

In [54]:
result = pd.DataFrame({'ImageId':list(range(1,test.shape[0]+1)),
                       'Label':pred})
result.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [55]:
result.to_csv('mlp_output.csv', index=False, header=True)
client_242955d4e4ca4cad87468d854b73c857.upload_file(Filename='output.csv', Bucket='test-donotdelete-pr-ectconhjpferec', Key='output.csv')

## Convolutional Neural Network

The MLP gives good accuracy, but is a generic model, not specialised for image data. Convolutional Neural Networks (CNNs) are specifically designed for processing grid-shaped data such as images, and should, theoretically, have much greater statistical efficiency when applied to this task, achieving better results.

In [ ]:
cnn = Sequential()
cnn.add(Conv2D(4, (3,3), padding='same', input_shape=(28,28,1)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(8, (3,3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(Conv2D(8, (3,3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Flatten())
cnn.add(Dense(32, activation='relu'))
cnn.add(Dense(10, activation='softmax'))
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_img_cnn = train_img.reshape(train_img.shape[0],28,28,1)

cnn_fit = cnn.fit(train_img_cnn, labels, epochs=100, batch_size=64, validation_split=0.1, callbacks=[early_stopping])

Train on 37800 samples, validate on 4200 samples
Epoch 1/100
37800/37800 [==============================] - 42s 1ms/step - loss: 2.5438 - acc: 0.5238 - val_loss: 0.4896 - val_acc: 0.8579
Epoch 2/100
37800/37800 [==============================] - 41s 1ms/step - loss: 0.4910 - acc: 0.8447 - val_loss: 0.2142 - val_acc: 0.9362
Epoch 3/100
  256/37800 [..............................] - ETA: 43s - loss: 0.3688 - acc: 0.8906